# Developing an AI application

Going forward, AI algorithms will be incorporated into more and more everyday applications. For example, you might want to include an image classifier in a smart phone app. To do this, you'd use a deep learning model trained on hundreds of thousands of images as part of the overall application architecture. A large part of software development in the future will be using these types of models as common parts of applications. 

In this project, you'll train an image classifier to recognize different species of flowers. You can imagine using something like this in a phone app that tells you the name of the flower your camera is looking at. In practice you'd train this classifier, then export it for use in your application. We'll be using [this dataset](http://www.robots.ox.ac.uk/~vgg/data/flowers/102/index.html) of 102 flower categories, you can see a few examples below. 

<img src='assets/Flowers.png' width=500px>

The project is broken down into multiple steps:

* Load and preprocess the image dataset
* Train the image classifier on your dataset
* Use the trained classifier to predict image content

We'll lead you through each part which you'll implement in Python.

When you've completed this project, you'll have an application that can be trained on any set of labeled images. Here your network will be learning about flowers and end up as a command line application. But, what you do with your new skills depends on your imagination and effort in building a dataset. For example, imagine an app where you take a picture of a car, it tells you what the make and model is, then looks up information about it. Go build your own dataset and make something new.

First up is importing the packages you'll need. It's good practice to keep all the imports at the beginning of your code. As you work through this notebook and find you need to import a package, make sure to add the import up here.

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

# Imports here
from dataloader import ImageDataLoader, BATCH_SIZE
from classifier import ImageClassifier, load_checkpoint
from helpers import visualization as viz
from helpers import torch_helpers as th
from helpers import utils

from predict import predict

import os

from PIL import Image
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# Getting an error with Jupyter Notebook Kernel shutting down. Running this cell fixes error. (Only happens when using matplotlib)
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

## Load the data

Here you'll use `torchvision` to load the data ([documentation](http://pytorch.org/docs/0.3.0/torchvision/index.html)). The data should be included alongside this notebook, otherwise you can [download it here](https://s3.amazonaws.com/content.udacity-data.com/nd089/flower_data.tar.gz). The dataset is split into three parts, training, validation, and testing. For the training, you'll want to apply transformations such as random scaling, cropping, and flipping. This will help the network generalize leading to better performance. You'll also need to make sure the input data is resized to 224x224 pixels as required by the pre-trained networks.

The validation and testing sets are used to measure the model's performance on data it hasn't seen yet. For this you don't want any scaling or rotation transformations, but you'll need to resize then crop the images to the appropriate size.

The pre-trained networks you'll use were trained on the ImageNet dataset where each color channel was normalized separately. For all three sets you'll need to normalize the means and standard deviations of the images to what the network expects. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`, calculated from the ImageNet images.  These values will shift each color channel to be centered at 0 and range from -1 to 1.


### Step 1: Loading Data
Please refer to "dataloader.py" for my implementation of a ImageDataLoader class, which I use to load all the data for this project.

In [ ]:
# Set data_dir (path to the main dataset folder)
data_dir = 'flowers'

# Init data loader
dataloader = ImageDataLoader(data_dir)

# DONE: Define your transforms for the training, validation, and testing sets
data_transforms = dataloader.data_transforms

# DONE: Load the datasets with ImageFolder
image_datasets = dataloader.image_datasets

# DONE: Using the image datasets and the trainforms, define the dataloaders
dataloaders = dataloader.dataloaders

#### Sanity Check #1: Checking if data loaded correctly

We plot some images from the training data loader, as well as some original and transformed images.

In [ ]:
# Show a single image from the validation dataloader
image, labels = dataloader.get_batch_data("train")
viz.show_image(image[0], title='Sample Image', normalize=True);

In [ ]:
# Plot some original and transformed images
# These are not from our dataloaders
# We just apply the same transform to these test images
image_paths = ["flowers/train/1/image_06737.jpg",
               "flowers/train/12/image_04000.jpg",
               "flowers/train/31/image_06899.jpg"
               ]

viz.plot_transformed_images(image_paths, data_transforms['train'])

#### Sanity Check #2: Loaded Data

Just get some basic info about the data loaded

In [ ]:
# Get basic information about loaded data
for key, value in dataloader.dataset_sizes.items():
    print(f"dataset '{key}', with size: {value}")

print()

# Information about data loaders
for key, value in dataloader.dataloaders.items():
    print(f"dataloader '{key}', with dataloader: {value}, size: {len(value)} batches of {BATCH_SIZE} images (~{(round(len(value) * BATCH_SIZE))})")

    print()

### Label mapping

You'll also need to load in a mapping from category label to category name. You can find this in the file `cat_to_name.json`. It's a JSON object which you can read in with the [`json` module](https://docs.python.org/2/library/json.html). This will give you a dictionary mapping the integer encoded categories to the actual names of the flowers.

In [ ]:
# Use the load_cat_to_name function from the ImageDataLoader class
dataloader.load_cat_to_name('cat_to_name.json')

# Check: print the loaded data
print(f"cat_to_name: {dataloader.cat_to_name}")

# Building and training the classifier

Now that the data is ready, it's time to build and train the classifier. As usual, you should use one of the pretrained models from `torchvision.models` to get the image features. Build and train a new feed-forward classifier using those features.

We're going to leave this part up to you. Refer to [the rubric](https://review.udacity.com/#!/rubrics/1663/view) for guidance on successfully completing this section. Things you'll need to do:

* Load a [pre-trained network](http://pytorch.org/docs/master/torchvision/models.html) (If you need a starting point, the VGG networks work great and are straightforward to use)
* Define a new, untrained feed-forward network as a classifier, using ReLU activations and dropout
* Train the classifier layers using backpropagation using the pre-trained network to get the features
* Track the loss and accuracy on the validation set to determine the best hyperparameters

We've left a cell open for you below, but use as many as you need. Our advice is to break the problem up into smaller parts you can run separately. Check that each part is doing what you expect, then move on to the next. You'll likely find that as you work through each part, you'll need to go back and modify your previous code. This is totally normal!

When training make sure you're updating only the weights of the feed-forward network. You should be able to get the validation accuracy above 70% if you build everything right. Make sure to try different hyperparameters (learning rate, units in the classifier, epochs, etc) to find the best model. Save those hyperparameters to use as default values in the next part of the project.

One last important tip if you're using the workspace to run your code: To avoid having your workspace disconnect during the long-running tasks in this notebook, please read in the earlier page in this lesson called Intro to
GPU Workspaces about Keeping Your Session Active. You'll want to include code from the workspace_utils.py module.

**Note for Workspace users:** If your network is over 1 GB when saved as a checkpoint, there might be issues with saving backups in your workspace. Typically this happens with wide dense layers after the convolutional layers. If your saved checkpoint is larger than 1 GB (you can open a terminal and check with `ls -lh`), you should reduce the size of your hidden layers and train again.

### Step 2: Build and Train Classifier

In this step, we use a custom ImageClassifier class, which basically uses transfer learning (pre-trained model). Refer to "classifier.py" for more details.

In [ ]:
# DONE: Build and train your network
# Init the ImageClassifier
image_classifier = ImageClassifier(n_classes=dataloader.num_classes)
# Init model and show model summary before setting the new classifier
image_classifier.init_model()
image_classifier.show_model_summary()

In [ ]:
# Set the classifier and show the new model
image_classifier.set_classifier()
image_classifier.show_model_summary()

In [ ]:
# Show the device info
image_classifier.show_device_info()

In [ ]:
results = image_classifier.train(dataloaders=dataloader.dataloaders, num_epochs=8)

In [ ]:
# Print the results
print(results)

#### Sanity Check #4: Plot Loss Curves

We plot the loss curves to check how our training performed

In [ ]:
viz.plot_loss_curves(results)

## Testing your network

It's good practice to test your trained network on test data, images the network has never seen either in training or validation. This will give you a good estimate for the model's performance on completely new images. Run the test images through the network and measure the accuracy, the same way you did validation. You should be able to reach around 70% accuracy on the test set if the model has been trained well.

In [ ]:
# DONE: Do validation on the test set
image_classifier.check_accuracy_on_test_data(dataloader=dataloader.dataloaders['test'])

## Save the checkpoint

Now that your network is trained, save the model so you can load it later for making predictions. You probably want to save other things such as the mapping of classes to indices which you get from one of the image datasets: `image_datasets['train'].class_to_idx`. You can attach this to the model as an attribute which makes inference easier later on.

```model.class_to_idx = image_datasets['train'].class_to_idx```

Remember that you'll want to completely rebuild the model later so you can use it for inference. Make sure to include any information you need in the checkpoint. If you want to load the model and keep training, you'll want to save the number of epochs as well as the optimizer state, `optimizer.state_dict`. You'll likely want to use this trained model in the next part of the project, so best to save it now.

In [ ]:
# Print the model state_dict_keys
print(f"Model State Dict Keys: {image_classifier.model.state_dict().keys()}")

In [ ]:
# Save the checkpoint using save_checkpoint method of ImageClassifier object
image_classifier.save_checkpoint("checkpoints/checkpoint.pth", dataloader=dataloader)

## Loading the checkpoint

At this point it's good to write a function that can load a checkpoint and rebuild the model. That way you can come back to this project and keep working on it without having to retrain the network.

In [ ]:
# DONE: Function written into ImageClassifier class

In [ ]:
# Load checkpoint into new ImageClassifier
new_classifier = load_checkpoint("checkpoints/checkpoint.pth")

# Show the model summary
new_classifier.show_model_summary()

In [ ]:
# Check the new classifier model state_dict keys and make sure all is good
print(f"Model State Dict Keys: {new_classifier.model.state_dict().keys()}")

# Inference for classification

Now you'll write a function to use a trained network for inference. That is, you'll pass an image into the network and predict the class of the flower in the image. Write a function called `predict` that takes an image and a model, then returns the top $K$ most likely classes along with the probabilities. It should look like 

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

First you'll need to handle processing the input image such that it can be used in your network. 

## Image Preprocessing

You'll want to use `PIL` to load the image ([documentation](https://pillow.readthedocs.io/en/latest/reference/Image.html)). It's best to write a function that preprocesses the image so it can be used as input for the model. This function should process the images in the same manner used for training. 

First, resize the images where the shortest side is 256 pixels, keeping the aspect ratio. This can be done with the [`thumbnail`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) or [`resize`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) methods. Then you'll need to crop out the center 224x224 portion of the image.

Color channels of images are typically encoded as integers 0-255, but the model expected floats 0-1. You'll need to convert the values. It's easiest with a Numpy array, which you can get from a PIL image like so `np_image = np.array(pil_image)`.

As before, the network expects the images to be normalized in a specific way. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`. You'll want to subtract the means from each color channel, then divide by the standard deviation. 

And finally, PyTorch expects the color channel to be the first dimension but it's the third dimension in the PIL image and Numpy array. You can reorder dimensions using [`ndarray.transpose`](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.ndarray.transpose.html). The color channel needs to be first and retain the order of the other two dimensions.

In [ ]:
# DONE: Function defined in helpers/visualization.py

To check your work, the function below converts a PyTorch tensor and displays it in the notebook. If your `process_image` function works, running the output through this function should return the original image (except for the cropped out portions).

In [ ]:
def imshow(image, ax=None, title=None):
    """Imshow for Tensor."""
    if ax is None:
        fig, ax = plt.subplots()
    
    # PyTorch tensors assume the color channel is the first dimension
    # but matplotlib assumes is the third dimension
    image = image.numpy().transpose((1, 2, 0))
    
    # Undo preprocessing
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = std * image + mean
    
    # Image needs to be clipped between 0 and 1 or it looks like noise when displayed
    image = np.clip(image, 0, 1)
    
    ax.imshow(image)
    
    return ax

In [ ]:
# test
rand_img_path = utils.get_random_image_path(os.path.join(data_dir, 'flowers', 'train'))
print(f"rand img path: ", rand_img_path)
img = viz.process_image(rand_img_path)
imshow(img);

In [ ]:
# try other function
img2 = viz.process_image_simple(rand_img_path, dataloader.data_transforms['test'])
imshow(img2);

## Class Prediction

Once you can get images in the correct format, it's time to write a function for making predictions with your model. A common practice is to predict the top 5 or so (usually called top-$K$) most probable classes. You'll want to calculate the class probabilities then find the $K$ largest values.

To get the top $K$ largest values in a tensor use [`x.topk(k)`](http://pytorch.org/docs/master/torch.html#torch.topk). This method returns both the highest `k` probabilities and the indices of those probabilities corresponding to the classes. You need to convert from these indices to the actual class labels using `class_to_idx` which hopefully you added to the model or from an `ImageFolder` you used to load the data ([see here](#Save-the-checkpoint)). Make sure to invert the dictionary so you get a mapping from index to class as well.

Again, this method should take a path to an image and a model checkpoint, then return the probabilities and classes.

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

In [ ]:
# DONE: Implement the code to predict the class from an image file
# Get a random image path
rand_img_path = utils.get_random_image_path(os.path.join(data_dir, 'flowers', 'train'))

# Make the prediction
device = th.get_device()
probs, classes = predict(rand_img_path, new_classifier.model, device)

# Print the results
print(probs)
print(classes)

## Sanity Checking

Now that you can use a trained model for predictions, check to make sure it makes sense. Even if the testing accuracy is high, it's always good to check that there aren't obvious bugs. Use `matplotlib` to plot the probabilities for the top 5 classes as a bar graph, along with the input image. It should look like this:

<img src='assets/inference_example.png' width=300px>

You can convert from the class integer encoding to actual flower names with the `cat_to_name.json` file (should have been loaded earlier in the notebook). To show a PyTorch tensor as an image, use the `imshow` function defined above.

In [ ]:
def show_class_prediction_figure(image_path, model, device, cat_to_name):
    probabilities, classes = predict(image_path, model, device)

    print("Probs: ", probabilities)
    print()
    print("Classes: ", classes)

    # To plot more than one sublot
    # I used https://github.com/udacity/DL_PyTorch/blob/master/helper.py
    # As a starting point.
    fig, (ax1, ax2) = plt.subplots(figsize=(6, 8), nrows=2)

    fig.subplots_adjust(hspace=0.1, wspace=0.1)

    # get the index of the maximum probability
    # https://docs.scipy.org/doc/numpy/reference/generated/numpy.argmax.html
    max_index = 0

    # get the label for the class with the max probability
    max_index_label = cat_to_name[str(classes.tolist()[0][0])]

    # We 'plot' the image
    # open the image
    image = Image.open(image_path)

    # turn axis off, set the title and show the image
    ax1.axis('off')
    ax1.set_title(max_index_label)
    ax1.imshow(image)

    # create all the labels by converting using the cat_to_name
    # mapping
    labels = []
    for cls in classes.tolist()[0]:
        # print("class:", cls)
        labels.append(cat_to_name[str(cls)])

    # show the bar graph
    y_pos = np.arange(len(labels))

    # print(f"y_pos: {y_pos}")

    ax2.barh(y_pos, probabilities.tolist()[0])
    ax2.set_yticks(y_pos)
    ax2.set_yticklabels(labels)
    ax2.set_xlim(0, 1.0)

    plt.tight_layout()


In [ ]:
# DONE: Display an image along with the top 5 classes
device = th.get_device()
# get a random image from a random path
random_img_path = utils.get_random_image_path(os.path.join(data_dir, 'flowers', 'test'))
print("predicting image: ", random_img_path)

show_class_prediction_figure(
    random_img_path,
    new_classifier.model,
    th.get_device(),
    dataloader.cat_to_name
)